## Project 2 work

In [9]:
project_id = "cs378-fa2025"
region = "us-central1"
bucket_name = "air-travel-open-data"
input_parent_folder = "initial-load"
output_parent_folder = "lakehouse"
dataset_name = "air_travel_raw"

In [4]:
!pip install gcsfs -q

In [11]:
import pandas

### Convert to parquet format and store them in the lakehouse area of the bucket

In [12]:
# on-time-performance
folder = "on-time-performance"
file_path = f"gs://{bucket_name}/{input_parent_folder}/{folder}/Airline_Delay_Cause.csv"
df = pandas.read_csv(file_path)
df.to_parquet(f"gs://{bucket_name}/{output_parent_folder}/{folder}/Airline_Delay_Cause.parquet")

In [36]:
# openflights
folder = "openflights"
data_files = ["airlines", "airports_ext", "countries", "planes", "routes"]

for data_file in data_files:
    file_path = f"gs://{bucket_name}/{input_parent_folder}/{folder}/{data_file}.csv"
    df = pandas.read_csv(file_path)
    df.to_parquet(f"gs://{bucket_name}/{output_parent_folder}/{folder}/{data_file}/{data_file}.parquet")

In [15]:
# our-airports
folder = "our-airports"
file_path = f"gs://{bucket_name}/{input_parent_folder}/{folder}/airport-comments.tsv"
df = pandas.read_csv(file_path, sep="\t")
df.to_parquet(f"gs://{bucket_name}/{output_parent_folder}/{folder}/airport-comments.parquet")

##### Create a connection resource in BQ. Follow the steps in [this tutorial](https://cloud.google.com/bigquery/docs/create-cloud-resource-connection).

In [16]:
!bq mk --connection --location=$region --project_id=$project_id --connection_type=CLOUD_RESOURCE cloud-storage-connection

Connection 988876466742.us-central1.cloud-storage-connection successfully created


In [19]:
!bq show --connection 988876466742.us-central1.cloud-storage-connection

Connection 988876466742.us-central1.cloud-storage-connection

                        name                          friendlyName   description    Last modified         type        hasCredential                                            properties                                            
 --------------------------------------------------- -------------- ------------- ----------------- ---------------- --------------- ----------------------------------------------------------------------------------------------- 
  988876466742.us-central1.cloud-storage-connection                                05 Sep 14:52:51   CLOUD_RESOURCE   False           {"serviceAccountId": "bqcx-988876466742-opo0@gcp-sa-bigquery-condel.iam.gserviceaccount.com"}  



##### Grant your service account the Storage Object Viewer role

In [26]:
!gcloud storage buckets add-iam-policy-binding gs://$bucket_name \
  --member=serviceAccount:bqcx-988876466742-opo0@gcp-sa-bigquery-condel.iam.gserviceaccount.com \
  --role=roles/storage.objectAdmin

bindings:
- members:
  - projectEditor:cs378-fa2025
  - projectOwner:cs378-fa2025
  role: roles/storage.legacyBucketOwner
- members:
  - projectViewer:cs378-fa2025
  role: roles/storage.legacyBucketReader
- members:
  - projectEditor:cs378-fa2025
  - projectOwner:cs378-fa2025
  role: roles/storage.legacyObjectOwner
- members:
  - projectViewer:cs378-fa2025
  role: roles/storage.legacyObjectReader
- members:
  - serviceAccount:bqcx-988876466742-opo0@gcp-sa-bigquery-condel.iam.gserviceaccount.com
  role: roles/storage.objectAdmin
- members:
  - serviceAccount:bqcx-988876466742-opo0@gcp-sa-bigquery-condel.iam.gserviceaccount.com
  role: roles/storage.objectViewer
etag: CAM=
kind: storage#policy
resourceId: projects/_/buckets/air-travel-open-data
version: 1


### Create and load the Iceberg tables

#### Create the BQ dataset for the raw data

In [23]:
%%bigquery
create schema if not exists `cs378-fa2025`.air_travel_raw
options (
    location = 'us-central1',
    description = 'dataset for the raw layer of the lakehouse')

Query is running:   0%|          |

""


#### flight_delays table

In [32]:
%%bigquery

CREATE OR REPLACE TABLE air_travel_raw.flight_delays
(
  year INT64,
  month INT64,
  carrier STRING,
  carrier_name STRING,
  airport STRING,
  airport_name STRING,
  arr_flights FLOAT64,
  arr_del15 FLOAT64,
  carrier_ct FLOAT64,
  weather_ct FLOAT64,
  nas_ct FLOAT64,
  security_ct FLOAT64,
  late_aircraft_ct FLOAT64,
  arr_cancelled FLOAT64,
  arr_diverted FLOAT64,
  arr_delay FLOAT64,
  carrier_delay FLOAT64,
  weather_delay FLOAT64,
  nas_delay FLOAT64,
  security_delay FLOAT64,
  late_aircraft_delay FLOAT64
)
WITH CONNECTION `988876466742.us-central1.cloud-storage-connection`
OPTIONS (
 file_format = 'PARQUET',
 table_format = 'ICEBERG',
 storage_uri = 'gs://air-travel-open-data/lakehouse/on-time-performance'
);

Query is running:   0%|          |

""


In [33]:
%%bigquery

load data into air_travel_raw.flight_delays
from files (
 format = 'parquet',
 uris = ['gs://air-travel-open-data/lakehouse/on-time-performance/Airline_Delay_Cause.parquet']);

Query is running:   0%|          |

""


In [35]:
%%bigquery

select * from air_travel_raw.flight_delays
limit 10

Query is running:   0%|          |

Downloading:   0%|          |

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2024,4,9E,Endeavor Air Inc.,ABE,"Allentown/Bethlehem/Easton, PA: Lehigh Valley ...",86.0,9.0,3.12,0.00,...,0.0,3.73,0.0,0.0,486.0,231.0,0.0,64.0,0.0,191.0
1,2024,4,9E,Endeavor Air Inc.,AEX,"Alexandria, LA: Alexandria International",60.0,9.0,2.53,0.00,...,0.0,2.70,0.0,1.0,603.0,185.0,0.0,173.0,0.0,245.0
2,2024,4,9E,Endeavor Air Inc.,AGS,"Augusta, GA: Augusta Regional at Bush Field",156.0,16.0,10.57,0.00,...,0.0,3.23,0.0,0.0,703.0,465.0,0.0,123.0,0.0,115.0
3,2024,4,9E,Endeavor Air Inc.,ALB,"Albany, NY: Albany International",115.0,16.0,8.00,0.98,...,0.0,4.31,0.0,0.0,865.0,517.0,41.0,88.0,0.0,219.0
4,2024,4,9E,Endeavor Air Inc.,ATL,"Atlanta, GA: Hartsfield-Jackson Atlanta Intern...",2032.0,241.0,60.87,12.49,...,0.0,110.66,3.0,1.0,16233.0,5737.0,1528.0,1745.0,0.0,7223.0
5,2024,4,9E,Endeavor Air Inc.,ATW,"Appleton, WI: Appleton International",20.0,1.0,0.00,0.00,...,0.0,0.00,0.0,0.0,18.0,0.0,0.0,18.0,0.0,0.0
6,2024,4,9E,Endeavor Air Inc.,AUS,"Austin, TX: Austin - Bergstrom International",77.0,11.0,4.59,0.00,...,0.0,2.28,0.0,0.0,588.0,336.0,0.0,100.0,0.0,152.0
7,2024,4,9E,Endeavor Air Inc.,AVL,"Asheville, NC: Asheville Regional",49.0,5.0,2.00,0.00,...,0.0,0.00,0.0,0.0,279.0,197.0,0.0,82.0,0.0,0.0
8,2024,4,9E,Endeavor Air Inc.,BDL,"Hartford, CT: Bradley International",56.0,6.0,2.03,0.00,...,0.0,0.85,1.0,0.0,213.0,82.0,0.0,92.0,0.0,39.0
9,2024,4,9E,Endeavor Air Inc.,BGM,"Binghamton, NY: Greater Binghamton/Edwin A. Li...",30.0,3.0,1.04,0.00,...,0.0,1.65,0.0,0.0,89.0,25.0,0.0,13.0,0.0,51.0


#### airlines table

In [40]:
%%bigquery

CREATE OR REPLACE TABLE air_travel_raw.airlines
(
  airline_id INT64,
  name STRING,
  alias STRING,
  iata STRING,
  icao STRING,
  callsign STRING,
  country STRING,
  active STRING
)
WITH CONNECTION `988876466742.us-central1.cloud-storage-connection`
OPTIONS (
 file_format = 'PARQUET',
 table_format = 'ICEBERG',
 storage_uri = 'gs://air-travel-open-data/lakehouse/openflights/airlines'
);

Query is running:   0%|          |

""


In [41]:
%%bigquery

load data into air_travel_raw.airlines
from files (
 format = 'parquet',
 uris = ['gs://air-travel-open-data/lakehouse/openflights/airlines/airlines.parquet']);

Query is running:   0%|          |

""


In [42]:
%%bigquery

select * from air_travel_raw.airlines
limit 10

Query is running:   0%|          |

Downloading:   0%|          |

,airline_id,name,alias,iata,icao,callsign,country,active
0,-1,Unknown,\N,-,None,\N,\N,Y
1,1,Private flight,\N,-,None,None,None,Y
2,2,135 Airways,\N,None,GNL,GENERAL,United States,N
3,3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y
4,4,2 Sqn No 1 Elementary Flying Training School,\N,None,WYT,None,United Kingdom,N
5,5,213 Flight Unit,\N,None,TFU,None,Russia,N
6,6,223 Flight Unit State Airline,\N,None,CHD,CHKALOVSK-AVIA,Russia,N
7,7,224th Flight Unit,\N,None,TTF,CARGO UNIT,Russia,N
8,8,247 Jet Ltd,\N,None,TWF,CLOUD RUNNER,United Kingdom,N
9,9,3D Aviation,\N,None,SEC,SECUREX,United States,N


#### airports table

In [52]:
%%bigquery

CREATE OR REPLACE TABLE air_travel_raw.airports
(
  airport_id INT64,
  airport_name STRING,
  airport_city STRING,
  airport_country STRING,
  iata STRING,
  icao STRING,
  latitude FLOAT64,
  longitude FLOAT64,
  altitude INT64,
  timezone STRING,
  daylight_savings_time STRING,
  tz_database_timezone STRING,
  type STRING,
  source STRING
)
WITH CONNECTION `988876466742.us-central1.cloud-storage-connection`
OPTIONS (
 file_format = 'PARQUET',
 table_format = 'ICEBERG',
 storage_uri = 'gs://air-travel-open-data/lakehouse/openflights/airports'
);

Query is running:   0%|          |

""


In [53]:
%%bigquery

load data into air_travel_raw.airports
from files (
 format = 'parquet',
 uris = ['gs://air-travel-open-data/lakehouse/openflights/airports_ext/airports_ext.parquet']);

Query is running:   0%|          |

""


In [55]:
%%bigquery

select * from air_travel_raw.airports
limit 10

Query is running:   0%|          |

Downloading:   0%|          |

,airport_id,airport_name,airport_city,airport_country,iata,icao,latitude,longitude,altitude,timezone,daylight_savings_time,tz_database_timezone,type,source
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports
5,6,Wewak International Airport,Wewak,Papua New Guinea,WWK,AYWK,-3.583830,143.669006,19,10,U,Pacific/Port_Moresby,airport,OurAirports
6,7,Narsarsuaq Airport,Narssarssuaq,Greenland,UAK,BGBW,61.160500,-45.425999,112,-3,E,America/Godthab,airport,OurAirports
7,8,Godthaab / Nuuk Airport,Godthaab,Greenland,GOH,BGGH,64.190903,-51.678101,283,-3,E,America/Godthab,airport,OurAirports
8,9,Kangerlussuaq Airport,Sondrestrom,Greenland,SFJ,BGSF,67.012222,-50.711603,165,-3,E,America/Godthab,airport,OurAirports
9,10,Thule Air Base,Thule,Greenland,THU,BGTL,76.531197,-68.703201,251,-4,E,America/Thule,airport,OurAirports


#### countries table

In [56]:
%%bigquery

CREATE OR REPLACE TABLE air_travel_raw.countries
(
  country_name STRING,
  iso_code STRING,
  dafif_code STRING
)
WITH CONNECTION `988876466742.us-central1.cloud-storage-connection`
OPTIONS (
 file_format = 'PARQUET',
 table_format = 'ICEBERG',
 storage_uri = 'gs://air-travel-open-data/lakehouse/openflights/countries'
);

Query is running:   0%|          |

""


In [57]:
%%bigquery

load data into air_travel_raw.countries
from files (
 format = 'parquet',
 uris = ['gs://air-travel-open-data/lakehouse/openflights/countries/countries.parquet']);

Query is running:   0%|          |

""


In [58]:
%%bigquery

select * from air_travel_raw.countries
limit 10

Query is running:   0%|          |

Downloading:   0%|          |

,country_name,iso_code,dafif_code
0,"Bonaire, Saint Eustatius and Saba",BQ,None
1,Aruba,AW,AA
2,Antigua and Barbuda,AG,AC
3,United Arab Emirates,AE,AE
4,Afghanistan,AF,AF
5,Algeria,DZ,AG
6,Azerbaijan,AZ,AJ
7,Albania,AL,AL
8,Armenia,AM,AM
9,Angola,AO,AO


#### aircrafts

In [59]:
%%bigquery

CREATE OR REPLACE TABLE air_travel_raw.aircrafts
(
  aircraft_name STRING,
  iata_code STRING,
  icao_code STRING
)
WITH CONNECTION `988876466742.us-central1.cloud-storage-connection`
OPTIONS (
 file_format = 'PARQUET',
 table_format = 'ICEBERG',
 storage_uri = 'gs://air-travel-open-data/lakehouse/openflights/aircrafts'
);

Query is running:   0%|          |

""


In [61]:
%%bigquery

load data into air_travel_raw.aircrafts
from files (
 format = 'parquet',
 uris = ['gs://air-travel-open-data/lakehouse/openflights/planes/planes.parquet']);

Query is running:   0%|          |

""


In [62]:
%%bigquery

select * from air_travel_raw.aircrafts
limit 10

Query is running:   0%|          |

Downloading:   0%|          |

,aircraft_name,iata_code,icao_code
0,Aerospatiale (Nord) 262,ND2,N262
1,Aerospatiale (Sud Aviation) Se.210 Caravelle,CRV,S210
2,Aerospatiale SN.601 Corvette,NDC,S601
3,Aerospatiale/Alenia ATR 42-300,AT4,AT43
4,Aerospatiale/Alenia ATR 42-500,AT5,AT45
5,Aerospatiale/Alenia ATR 42-600,ATR,AT46
6,Aerospatiale/Alenia ATR 72,AT7,AT72
7,Airbus A300,AB3,A30B
8,Airbus A300-600,AB6,A306
9,Airbus A300-600ST Super Transporter / Beluga,ABB,A3ST


#### flight_routes

In [69]:
%%bigquery

CREATE OR REPLACE TABLE air_travel_raw.flight_routes
(
  airline_code STRING,
  airline_id STRING,
  source_airport STRING,
  source_airport_id STRING,
  dest_airport STRING,
  dest_airport_id STRING,
  codeshare STRING,
  stops INT64,
  equipment STRING
)
WITH CONNECTION `988876466742.us-central1.cloud-storage-connection`
OPTIONS (
 file_format = 'PARQUET',
 table_format = 'ICEBERG',
 storage_uri = 'gs://air-travel-open-data/lakehouse/openflights/flight_routes'
);

Query is running:   0%|          |

""


In [70]:
%%bigquery

load data into air_travel_raw.flight_routes
from files (
 format = 'parquet',
 uris = ['gs://air-travel-open-data/lakehouse/openflights/routes/routes.parquet']);

Query is running:   0%|          |

""


In [71]:
%%bigquery

select * from air_travel_raw.flight_routes
limit 10

Query is running:   0%|          |

Downloading:   0%|          |

,airline_code,airline_id,source_airport,source_airport_id,dest_airport,dest_airport_id,codeshare,stops,equipment
0,2B,410,AER,2965,KZN,2990,None,0,CR2
1,2B,410,ASF,2966,KZN,2990,None,0,CR2
2,2B,410,ASF,2966,MRV,2962,None,0,CR2
3,2B,410,CEK,2968,KZN,2990,None,0,CR2
4,2B,410,CEK,2968,OVB,4078,None,0,CR2
5,2B,410,DME,4029,KZN,2990,None,0,CR2
6,2B,410,DME,4029,NBC,6969,None,0,CR2
7,2B,410,DME,4029,TGK,\N,None,0,CR2
8,2B,410,DME,4029,UUA,6160,None,0,CR2
9,2B,410,EGO,6156,KGD,2952,None,0,CR2


#### airport_reviews

In [77]:
%%bigquery

CREATE OR REPLACE TABLE air_travel_raw.airport_reviews
(
  id INT64,
  threadRef FLOAT64,
  airportRef INT64,
  airportIdent STRING,
  date STRING,
  memberNickname STRING,
  subject STRING,
  body STRING
)
WITH CONNECTION `988876466742.us-central1.cloud-storage-connection`
OPTIONS (
 file_format = 'PARQUET',
 table_format = 'ICEBERG',
 storage_uri = 'gs://air-travel-open-data/lakehouse/our-airports'
);

Query is running:   0%|          |

""


In [78]:
%%bigquery

load data into air_travel_raw.airport_reviews
from files (
 format = 'parquet',
 uris = ['gs://air-travel-open-data/lakehouse/our-airports/airport-comments.parquet']);

Query is running:   0%|          |

""


In [80]:
%%bigquery

select * from air_travel_raw.airport_reviews
limit 10

Query is running:   0%|          |

Downloading:   0%|          |

,id,threadRef,airportRef,airportIdent,date,memberNickname,subject,body
0,537583,82359.0,537582,VN-0055,2024-07-15 13:52:51,kubet77meme,KUBET77 MEME,"Kubet là nhà cái uy tín bậc nhất Việt Nam, tr..."
1,537399,82358.0,31705,ZSQZ,2024-07-14 4:34:25,malaybear,(no subject),welcome to my favorite airport.
2,537260,82357.0,336969,US-1936,2024-07-12 19:48:02,Dmiddlet26,re: runway still maintained,Reply to @Dmiddlet26:
3,537254,82357.0,336969,US-1936,2024-07-12 16:28:00,Dmiddlet26,runway still maintained,just had a mini fly in there a few weekends ag...
4,537253,81417.0,336969,US-1936,2024-07-12 16:27:15,Dmiddlet26,re: Fun place to land,Reply to @Dmiddlet26:
5,537241,82356.0,537240,GB-1132,2024-07-12 11:28:38,ukairporttaxi,Link Building,We at UK Airport Taxi aim to provide the most ...
6,536849,82355.0,26380,UTAA,2024-07-09 14:02:29,jansmit74,poort naar de hel,Ik zoek de poort naar de hel die moet hier in ...
7,536444,82354.0,45063,SCMR,2024-07-06 17:51:10,jole,(no subject),Is this a duplicate of SCGL? What is the prope...
8,536359,82353.0,323478,RU-0470,2024-07-06 4:00:31,Bohaska,Russian Wikipedia URL,This airport doesn't have any English sources ...
9,535994,82352.0,25008,TN94,2024-07-03 22:20:25,animebirder,Is this actually still open?,"It's still listed on AirNav, but there's a gol..."


### Add table annotations (_data_source and _load_time)

#### flight_delays

In [88]:
%%bigquery

alter table air_travel_raw.flight_delays
    add column _data_source STRING, add column _load_time TIMESTAMP;

Query is running:   0%|          |

""


In [90]:
%%bigquery

update air_travel_raw.flight_delays
    set _data_source = 'on-time-performance', _load_time = current_timestamp()
    where 1 = 1

Query is running:   0%|          |

""


In [92]:
%%bigquery

select * from air_travel_raw.flight_delays
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,_data_source,_load_time
0,2024,3,AS,Alaska Airlines Network,KOA,"Kona, HI: Ellison Onizuka Kona International a...",192.0,26.0,6.40,0.00,...,0.0,1.0,1062.0,304.0,0.0,521.0,0.0,237.0,on-time-performance,2025-09-05 17:00:47.734796+00:00
1,2024,2,9E,Endeavor Air Inc.,PNS,"Pensacola, FL: Pensacola International",33.0,3.0,1.45,0.00,...,1.0,0.0,240.0,93.0,0.0,0.0,0.0,147.0,on-time-performance,2025-09-05 17:00:47.734796+00:00
2,2024,2,NK,Spirit Airlines,RDU,"Raleigh/Durham, NC: Raleigh-Durham International",21.0,4.0,0.88,0.00,...,0.0,0.0,302.0,69.0,0.0,113.0,0.0,120.0,on-time-performance,2025-09-05 17:00:47.734796+00:00
3,2024,2,AS,Alaska Airlines Network,DTW,"Detroit, MI: Detroit Metro Wayne County",25.0,3.0,0.00,0.00,...,0.0,0.0,53.0,0.0,0.0,53.0,0.0,0.0,on-time-performance,2025-09-05 17:00:47.734796+00:00
4,2024,1,AS,Alaska Airlines Network,SLC,"Salt Lake City, UT: Salt Lake City International",104.0,37.0,13.41,0.25,...,14.0,0.0,2608.0,895.0,14.0,166.0,0.0,1533.0,on-time-performance,2025-09-05 17:00:47.734796+00:00


#### airlines

In [93]:
%%bigquery

alter table air_travel_raw.airlines
    add column _data_source STRING, add column _load_time TIMESTAMP;

Query is running:   0%|          |

""


In [94]:
%%bigquery

update air_travel_raw.airlines
    set _data_source = 'openflights', _load_time = current_timestamp()
    where 1 = 1

Query is running:   0%|          |

""


In [95]:
%%bigquery

select * from air_travel_raw.airlines
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,airline_id,name,alias,iata,icao,callsign,country,active,_data_source,_load_time
0,102,Aeroenlaces Nacionales,\N,None,AEN,None,Mexico,N,openflights,2025-09-05 17:02:52.385895+00:00
1,128,Alliance,\N,None,AFJ,JAMBO,Uganda,N,openflights,2025-09-05 17:02:52.385895+00:00
2,147,Air-Angol,\N,None,NGO,AIR ANGOL,Angola,N,openflights,2025-09-05 17:02:52.385895+00:00
3,257,Aktjubavia,\N,None,AKB,KARAB,Kazakhstan,N,openflights,2025-09-05 17:02:52.385895+00:00
4,270,Aeroalas Colombia,\N,None,ALE,AEROALAS,Colombia,N,openflights,2025-09-05 17:02:52.385895+00:00


#### airports

In [97]:
%%bigquery

alter table air_travel_raw.airports
    add column _data_source STRING, add column _load_time TIMESTAMP;

Query is running:   0%|          |

""


In [98]:
%%bigquery

update air_travel_raw.airports
    set _data_source = 'openflights', _load_time = current_timestamp()
    where 1 = 1

Query is running:   0%|          |

""


In [99]:
%%bigquery

select * from air_travel_raw.airports
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,airport_id,airport_name,airport_city,airport_country,iata,icao,latitude,longitude,altitude,timezone,daylight_savings_time,tz_database_timezone,type,source,_data_source,_load_time
0,50,Arviat Airport,Eskimo Point,Canada,YEK,CYEK,61.094200,-94.070801,32,-6,A,America/Winnipeg,airport,OurAirports,openflights,2025-09-05 17:06:39.719594+00:00
1,80,Schefferville Airport,Schefferville,Canada,YKL,CYKL,54.805302,-66.805298,1709,-5,A,America/Toronto,airport,OurAirports,openflights,2025-09-05 17:06:39.719594+00:00
2,120,Regina International Airport,Regina,Canada,YQR,CYQR,50.431900,-104.666000,1894,-6,N,America/Regina,airport,OurAirports,openflights,2025-09-05 17:06:39.719594+00:00
3,153,Val-d'Or Airport,Val D'or,Canada,YVO,CYVO,48.053299,-77.782799,1107,-5,A,America/Toronto,airport,OurAirports,openflights,2025-09-05 17:06:39.719594+00:00
4,215,Ain Oussera Airport,Ain Oussera,Algeria,\N,DAAQ,35.525398,2.878710,2132,1,N,Africa/Algiers,airport,OurAirports,openflights,2025-09-05 17:06:39.719594+00:00


#### countries

In [100]:
%%bigquery

alter table air_travel_raw.countries
    add column _data_source STRING, add column _load_time TIMESTAMP;

Query is running:   0%|          |

""


In [101]:
%%bigquery

update air_travel_raw.countries
    set _data_source = 'openflights', _load_time = current_timestamp()
    where 1 = 1

Query is running:   0%|          |

""


In [102]:
%%bigquery

select * from air_travel_raw.countries
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,country_name,iso_code,dafif_code,_data_source,_load_time
0,Argentina,AR,AR,openflights,2025-09-05 17:07:49.475555+00:00
1,Belarus,BY,BO,openflights,2025-09-05 17:07:49.475555+00:00
2,Eritrea,ER,ER,openflights,2025-09-05 17:07:49.475555+00:00
3,Guernsey,GG,GK,openflights,2025-09-05 17:07:49.475555+00:00
4,India,IN,IN,openflights,2025-09-05 17:07:49.475555+00:00


#### aircrafts

In [87]:
%%bigquery

alter table air_travel_raw.aircrafts
    add column _data_source STRING, add column _load_time TIMESTAMP;

Query is running:   0%|          |

""


In [103]:
%%bigquery

update air_travel_raw.aircrafts
    set _data_source = 'openflights', _load_time = current_timestamp()
    where 1 = 1

Query is running:   0%|          |

""


In [104]:
%%bigquery

select * from air_travel_raw.aircrafts
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,aircraft_name,iata_code,icao_code,_data_source,_load_time
0,Aerospatiale/Alenia ATR 42-600,ATR,AT46,openflights,2025-09-05 17:08:28.551688+00:00
1,Airbus A319neo,31N,A19N,openflights,2025-09-05 17:08:28.551688+00:00
2,Antonov AN-140,A40,A140,openflights,2025-09-05 17:08:28.551688+00:00
3,Bombardier BD-100 Challenger 300,\N,CL30,openflights,2025-09-05 17:08:28.551688+00:00
4,British Aerospace 125-1000 series / Hawker/Ray...,H25,H25C,openflights,2025-09-05 17:08:28.551688+00:00


#### flight_routes

In [105]:
%%bigquery

alter table air_travel_raw.flight_routes
    add column _data_source STRING, add column _load_time TIMESTAMP;

Query is running:   0%|          |

""


In [106]:
%%bigquery

update air_travel_raw.flight_routes
    set _data_source = 'our-airports', _load_time = current_timestamp()
    where 1 = 1

Query is running:   0%|          |

""


In [107]:
%%bigquery

select * from air_travel_raw.flight_routes
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,airline_code,airline_id,source_airport,source_airport_id,dest_airport,dest_airport_id,codeshare,stops,equipment,_data_source,_load_time
0,2I,8359,LIM,2789,AYP,2786,None,0,142,our-airports,2025-09-05 17:10:02.704144+00:00
1,2L,2750,LCA,1197,ZRH,1678,None,0,319,our-airports,2025-09-05 17:10:02.704144+00:00
2,2P,897,MNL,2397,SUG,6016,None,0,DH4,our-airports,2025-09-05 17:10:02.704144+00:00
3,2Z,1729,BRA,7373,BSB,2531,None,0,AT7,our-airports,2025-09-05 17:10:02.704144+00:00
4,2Z,1729,PLU,2528,RAO,2613,None,0,AT7,our-airports,2025-09-05 17:10:02.704144+00:00


#### airport_reviews

In [81]:
%%bigquery

alter table air_travel_raw.airport_reviews
    add column _data_source STRING, add column _load_time TIMESTAMP;

Query is running:   0%|          |

""


In [83]:
%%bigquery

update air_travel_raw.airport_reviews
    set _data_source = 'our-airports', _load_time = current_timestamp()
    where 1 = 1

Query is running:   0%|          |

""


In [96]:
%%bigquery

select * from air_travel_raw.airport_reviews
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,id,threadRef,airportRef,airportIdent,date,memberNickname,subject,body,_data_source,_load_time
0,521558,82282.0,2212,EDDF,2024-02-26 12:18:37,E.M,Lighting system and signs for EDDF,could someone help me and tell me where I coul...,our-airports,2025-09-05 16:57:17.239377+00:00
1,514094,82222.0,323797,EBDR,2023-10-30 11:59:42,animebirder,Heliport oddity!,Evidently a heliport aboard a stationary boat ...,our-airports,2025-09-05 16:57:17.239377+00:00
2,511417,82186.0,46563,IN-0040,2023-08-27 4:14:39,pollenflysimulations,"Pedong is in West Bengal, India",Pedong is listed here erroneously under list o...,our-airports,2025-09-05 16:57:17.239377+00:00
3,511052,82176.0,511051,IT-0718,2023-08-12 9:42:59,Pugliapilot,Campo di volo Vicari,i not have much information regard this camp f...,our-airports,2025-09-05 16:57:17.239377+00:00
4,509732,82133.0,509439,CA-0876,2023-06-24 18:51:57,dsdaddad3,sorry,I AM very sorry of confusing that airstrip wit...,our-airports,2025-09-05 16:57:17.239377+00:00
